<a href="https://colab.research.google.com/github/lbolsinger/BlackJack_ICS3U/blob/main/scipy_minimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Imports

from scipy.optimize import minimize
import unittest
import numpy as np


## Expected input

'''
X_A_B
given = [000, 001, 010, 011,
         100, 101, 110, 111]

X_A_A#_B
vars = [0000, 0001, 0010, 0011,
        0100, 0101, 0110, 0111,
        1000, 1001, 1010, 1011,
        1100, 1101, 1110, 1111]
'''


## Data

GIVEN = [(1+(-1)**(0+0+0*0)/(2**(0.5)))/8,
         (1+(-1)**(0+1+0*0)/(2**(0.5)))/8,
         (1+(-1)**(1+0+1*0)/(2**(0.5)))/8,
         (1+(-1)**(1+1+1*0)/(2**(0.5)))/8,
         (1+(-1)**(0+0+0*1)/(2**(0.5)))/8,
         (1+(-1)**(0+1+0*1)/(2**(0.5)))/8,
         (1+(-1)**(1+0+1*1)/(2**(0.5)))/8,
         (1+(-1)**(1+1+1*1)/(2**(0.5)))/8]


## Index constants

# indices where the given variable is equal to one based on the vars order
X_EQUALS_1 = [8, 9, 10, 11, 12, 13, 14, 15]
A_EQUALS_1 = [4, 5, 6, 7, 12, 13, 14, 15]
A_SHARP_EQUALS_1 = [2, 3, 6, 7, 10, 11, 14, 15]
B_EQUALS_1 = [1, 3, 5, 7, 9, 11, 13, 15]


## Calculations

def As(vars, a):
  # p'(A# = a)
  p_a = 0
  for i in range(16):
    if (i in A_SHARP_EQUALS_1) == a:
      p_a += vars[i]
  return p_a

def As_B(vars, a, b):
  # p'(A# = a, B = b)
  p_ab = 0
  for i in range(16):
    if ((i in A_SHARP_EQUALS_1) == a and (i in B_EQUALS_1) == b):
      p_ab += vars[i]
  return p_ab

def B_given_As(vars, a, b):
  # p'(B = b | A# = a)
  return As_B(vars, a, b)/As(vars, a)

def X(vars, x):
  p_x = 0
  for i in range(16):
    if (i in X_EQUALS_1) == x:
      p_x += vars[i]
  return p_x

def X_As(vars, x, a):
  # p'(X = x, A# = a)
  p_xa = 0
  for i in range(16):
    if ((i in X_EQUALS_1) == x and (i in A_SHARP_EQUALS_1) == a):
      p_xa += vars[i]
  return p_xa

def X_Given_As(vars, x, a):
  # p'(X = x | A# = a);
  return X_As(vars, x, a)/As(vars, a)

def X_As_B(vars, x, a, b):
  # p'(X = x, A# = a, B = b)
  p_xab = 0
  for i in range(16):
    if ((i in X_EQUALS_1 == x) and (i in A_SHARP_EQUALS_1) == a and (i in B_EQUALS_1) == b):
      p_xab += vars[i]
  return p_xab

def X_B_given_As(vars, x, a, b):
  # p'(X = x, B = b | A# = a)
  return X_As_B(vars, x, a, b)/As(vars, a)

def X_A_B(vars, x, a, b):
  # P(X = x, A = a, B = b)
  p_xab = 0
  for i in range(16):
    if ((i in X_EQUALS_1) == x and (i in A_EQUALS_1) == a and (i in B_EQUALS_1) == b):
      p_xab += vars[i]
  return p_xab

def X_A_As_B(vars, x, a, b, asharp):
  # p'(X = x, A = a, A# = asharp, B = b)
  p_xaab = 0
  for i in range(16):
    if ((i in X_EQUALS_1) == x and (i in A_EQUALS_1) == a and (i in B_EQUALS_1) == b and (i in A_SHARP_EQUALS_1) == asharp):
      p_xaab += vars[i]
  return p_xaab

def X_A_B_given_As(vars, x, a, b, asharp):
  # p'(X = x, A = a, B = b | A# = asharp)
  return X_A_As_B(vars, x, a, b, asharp)/As(vars, asharp)


## Objective

def objective(vars):
  return B_given_As(vars, 0, 0)


## Constraint helpers

def normalization_of_As(vars):
  # p'(A# = 0) + p'(A# = 1) = 1
  return As(vars, 0) + As(vars, 1) - 1

def normalization_over_As(vars, a):
  # Σ p'(XAB | A# = a) * p'(A# = a) = 1
  tot = 0
  for i in range(2):
    for j in range(2):
      for k in range(2):
        tot += X_A_B_given_As(vars, i, k, j, a)*As(vars, a)
  return tot - 1

def X_dsep_B_given_As(vars, x, b, a):
  # p'(X = x, B = b | A# = a) = p'(X = x | A#  = a) * p'(B = b | A# = a)
  # p'(X = x, B = b | A# = a) = p'(X = x) * p'(B = b | A# = a)
  return X_B_given_As(vars, x, a, b) - X(vars, x)*B_given_As(vars, a, b)

def trivial_inequality(vars, b, a):
  return B_given_As(vars, a, b) - As_B(vars, a, b)

def consistency(vars, x, a, asharp, b):
  # P(X = x, A = a, B = b | A# = asharp) = p'(X = x, A = a, B = b)
  return X_A_B_given_As(vars, x, a, b, asharp) - X_A_B(vars, x, a, b)

def test_con(vars):
  return X_A_B_given_As(vars, 0, 0, 0, 0) - 0.5

def test_con2(vars):
  return X_A_B_given_As(vars, 1, 1, 1, 1) - 0.5

## Constraints
constraints = []
constraints.append({'type': 'eq', 'fun': test_con})
constraints.append({'type': 'eq', 'fun': test_con2})
'''
constraints.append({'type': 'eq', 'fun': (lambda vars: normalization_of_As(vars))})
'''
constraints.append({'type': 'eq', 'fun':
 (lambda vars: normalization_over_As(vars, 0))})
constraints.append({'type': 'eq', 'fun':
 (lambda vars: normalization_over_As(vars, 1))})

for j in range(2):
  for k in range(2):
    for l in range(2):
      constraints.append({'type': 'eq', 'fun':
      (lambda vars: X_dsep_B_given_As(vars, j, k, l))})
'''
for m in range(2):
  for n in range(2):
    constraints.append({'type': 'ineq', 'fun':
    (lambda vars: trivial_inequality(vars, m, n))})

for o in range(2):
  for p in range(2):
    for q in range(2):
      constraints.append({'type': 'eq', 'fun':
      (lambda vars: consistency(vars, o, p, p, q))}) '''

bounds = [(0, 1)]*16


## Results

initial_guess = [0.07]*16

result = minimize(objective, initial_guess, method='SLSQP',
                  bounds=bounds, constraints=constraints)
print()
print(GIVEN)
print(result)
vals = result.x

# Nice printing
col_widths = [0] * 4
for i in range(4):
  for j in range(4):
    formatted_num = f"{vals[i*4 + j]:.3g}"
    col_widths[j] = max(col_widths[j], len(formatted_num))
for i in range(4):
  print()
  for j in range(4):
    formatted_num = f"{vals[i*4 + j]:.3g}"
    print(f"{formatted_num:<{col_widths[j] + 2}}", end="")

## Tests

class TestCalculations(unittest.TestCase):

  def setUp(self):
    # Define a sample 'vars' for testing
    self.vars = np.array([0.10, 0.20, 0.05, 0.15,
                          0.02, 0.08, 0.03, 0.17,
                          0.01, 0.04, 0.02, 0.03,
                          0.03, 0.06, 0.04, 0.07])

    # Define the index constants
    self.X_EQUALS_1 = [8, 9, 10, 11, 12, 13, 14, 15]
    self.A_EQUALS_1 = [4, 5, 6, 7, 12, 13, 14, 15]
    self.A_SHARP_EQUALS_1 = [2, 3, 6, 7, 10, 11, 14, 15]
    self.B_EQUALS_1 = [1, 3, 5, 7, 9, 11, 13, 15]

  def test_As(self):
    # Test P(A# = 0)
    self.assertAlmostEqual(As(self.vars, 0), 0.54, places=3)
    # Test P(A# = 1)
    self.assertAlmostEqual(As(self.vars, 1), 0.56, places=3)

  def test_As_B(self):
    # Test P(A# = 0, B = 0)
    self.assertAlmostEqual(As_B(self.vars, 0, 0), 0.16, places=3)
    # Test P(A# = 1, B = 1)
    self.assertAlmostEqual(As_B(self.vars, 1, 1), 0.42, places=3)

  def test_B_given_As(self):
    # Test P(B = 1 | A# = 0)
    self.assertAlmostEqual(B_given_As(self.vars, 0, 1), 0.38/0.54, places=3)
    # Test P(B = 0 | A# = 1)
    self.assertAlmostEqual(B_given_As(self.vars, 1, 0), 0.14/0.56, places=3)

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK



[0.21338834764831843, 0.03661165235168157, 0.03661165235168157, 0.21338834764831843, 0.21338834764831843, 0.03661165235168157, 0.21338834764831843, 0.03661165235168157]
 message: Singular matrix C in LSQ subproblem
 success: False
  status: 6
     fun: 0.5
       x: [ 7.000e-02  7.000e-02 ...  7.000e-02  7.000e-02]
     nit: 1
     jac: [ 8.929e-01 -8.929e-01 ...  0.000e+00  0.000e+00]
    nfev: 17
    njev: 1

0.07  0.07  0.07  0.07  
0.07  0.07  0.07  0.07  
0.07  0.07  0.07  0.07  
0.07  0.07  0.07  0.07  

In [ ]:
from scipy.optimize import minimize

'''
[000, 001, 010, 011,
 100, 101, 110, 111]
'''

dist_xab = [0.21338834764831843, 0.03661165235168157, 0.03661165235168157, 0.21338834764831843,
            0.21338834764831843, 0.03661165235168157, 0.21338834764831843, 0.03661165235168157]

dist_ab_given_x = [[],[]]
for i in range(4):
  dist_ab_given_x[0].append(dist_xab[i]/sum(dist_xab[0:4]))
  dist_ab_given_x[1].append(dist_xab[i+4]/sum(dist_xab[0:4]))

'''
[0000, 0001, 0010, 0011,
 0100, 0101, 0110, 0111,
 1000, 1001, 1010, 1011,
 1100, 1101, 1110, 1111]
'''

def get_index_xaab(x, a, a_s, b):
  return 8*x + 4*a + 2*a_s + b

def get_index_xab(x, a, b):
  return 4*x + 2*a + b

def a_sharp(vars, a_s):
  p_a_s = 0
  for i in range(2):
    for j in range(2):
      for k in range(2):
          p_a_s += vars[get_index_xaab(i, j, a_s, k)]
  return p_a_s

def x_a_as_b(vars, x, a, a_s, b):
  return vars[get_index_xaab(x, a, a_s, b)]

def b_X_a_sharp(vars, x, a_s, b):
  p_b_x_a_s = 0
  for i in range(2):
    p_b_x_a_s += vars[get_index_xaab(x, i, a_s, b)]
  return p_b_x_a_s

def x_a_sharp(vars, x, a_s):
  p_x_a_s = 0
  for i in range(2):
    for j in range(2):
      p_x_a_s += vars[get_index_xaab(x, i, a_s, j)]
  return p_x_a_s

def b_given_x_a_sharp(vars, x, a_s, b):
  return b_X_a_sharp(vars, x, a_s, b) / x_a_sharp(vars, x, a_s)

def a_sharp_b(vars, a_s, b):
  p_a_s_b = 0
  for i in range(2):
    for j in range(2):
      p_a_s_b += vars[get_index_xaab(i, j, a_s, b)]
  return p_a_s_b

def b_given_a_sharp(vars, a_s, b):
  return a_sharp_b(vars, a_s, b) / a_sharp(vars, a_s)

def consistency(vars, x, a, b):
  return vars[get_index_xaab(x, a, a, b)] - dist_xab[get_index_xab(x, a, b)] * a_sharp(vars, a)

def d_sep(vars, x, a_s, b):
  return b_given_x_a_sharp(vars, x, a_s, b) - b_given_a_sharp(vars, a_s, b)

def normalization(vars, a_s):
  return a_sharp(vars, a_s) - 1

def objective(vars):
  return b_given_a_sharp(vars, 1, 1)

constraints = []
'''
for i in range(2):
  constraints.append({'type': 'eq', 'fun': (lambda vars, a_s=i: normalization(vars, a_s))})

for j in range(2):
  for k in range(2):
    for l in range(2):
      constraints.append({'type': 'eq', 'fun': (lambda vars, x=j, a_s=k, b=l: d_sep(vars, x, a_s, b))})

for m in range(2):
  for n in range(2):
    for o in range(2):
      constraints.append({'type': 'eq', 'fun': (lambda vars, x=m, a=n, b=o: consistency(vars, x, a, b))})
'''
# Only 1 normalization constraint
constraints.append({'type': 'eq', 'fun': (lambda vars, a_s=1: normalization(vars, a_s))})

# 4 d-separation constraints
for x in range(2):
    for b in range(2):
        constraints.append({'type': 'eq', 'fun': (lambda vars, x=x, a_s=1, b=b: d_sep(vars, x, a_s, b))})

# 8 consistency constraints
for x in range(2):
    for a in range(2):
        for b in range(2):
            constraints.append({'type': 'eq', 'fun': (lambda vars, x=x, a=a, b=b: consistency(vars, x, a, b))})

initial = [1/16] * 16
bounds = [(0, 1)] * 16
result = minimize(objective, initial, method='SLSQP', constraints=constraints, bounds=bounds)
print(result)


 message: Singular matrix C in LSQ subproblem
 success: False
  status: 6
     fun: 0.5
       x: [ 6.250e-02  6.250e-02 ...  6.250e-02  6.250e-02]
     nit: 1
     jac: [ 0.000e+00  0.000e+00 ... -1.000e+00  1.000e+00]
    nfev: 17
    njev: 1


In [ ]:
from scipy.optimize import minimize
from functools import partial

# Original distribution over XAB
dist_xab = [0.21338834764831843, 0.03661165235168157, 0.03661165235168157, 0.21338834764831843,
            0.21338834764831843, 0.03661165235168157, 0.21338834764831843, 0.03661165235168157]

# Helper functions to get indices
def get_index_xaab(x, a, a_s, b):
    return 8 * x + 4 * a + 2 * a_s + b

def get_index_xab(x, a, b):
    return 4 * x + 2 * a + b

# Probabilities and marginalization helpers
def a_sharp(vars, a_s):
    return sum(vars[get_index_xaab(x, a, a_s, b)] for x in range(2) for a in range(2) for b in range(2))

def b_X_a_sharp(vars, x, a_s, b):
    return sum(vars[get_index_xaab(x, a, a_s, b)] for a in range(2))

def x_a_sharp(vars, x, a_s):
    return sum(vars[get_index_xaab(x, a, a_s, b)] for a in range(2) for b in range(2))

def a_b(vars, a, b):
    return sum(vars[get_index_xaab(x, a, a_s, b)] for x in range(2) for a_s in range(2))

def b_given_x_a_sharp(vars, x, a_s, b):
    denom = x_a_sharp(vars, x, a_s)
    return b_X_a_sharp(vars, x, a_s, b) / denom if denom > 1e-8 else 0

def a_sharp_b(vars, a_s, b):
    return sum(vars[get_index_xaab(x, a, a_s, b)] for x in range(2) for a in range(2))

def b_given_a_sharp(vars, a_s, b):
    denom = a_sharp(vars, a_s)
    return a_sharp_b(vars, a_s, b) / denom if denom > 1e-8 else 0

def consistency(vars, x, a, b):
    return vars[get_index_xaab(x, a, a, b)] - dist_xab[get_index_xab(x, a, b)] * a_sharp(vars, a)

def d_sep(vars, x, a_s, b):
    return b_given_x_a_sharp(vars, x, a_s, b) - b_given_a_sharp(vars, a_s, b)

def normalization(vars, a_s):
    return a_sharp(vars, a_s) - 1

def normalize_a_sharp(vars):
    return a_sharp(vars, 0) + a_sharp(vars, 1) - 1

def objective(vars):
    return b_given_a_sharp(vars, 0, 0)

# Constraint construction using partial
def make_dsep_constraint(x, a_s, b):
    return {'type': 'eq', 'fun': partial(d_sep, x=x, a_s=a_s, b=b)}

def make_consistency_constraint(x, a, b):
    return {'type': 'eq', 'fun': partial(consistency, x=x, a=a, b=b)}

def make_normalization_constraint(a_s):
    return {'type': 'eq', 'fun': partial(normalization, a_s=a_s)}

# Construct constraint list
constraints = [make_normalization_constraint(1), make_normalization_constraint(0)]
#constraints = [{'type': 'eq', 'fun': normalize_a_sharp}]
'''
for x in range(2):
    for a in range(2):
        for b in range(2):
            constraints.append(make_dsep_constraint(x, a, b))
'''
constraints.append(make_dsep_constraint(0, 0, 0))
constraints.append(make_dsep_constraint(0, 0, 1))
constraints.append(make_dsep_constraint(0, 1, 0))
constraints.append(make_dsep_constraint(0, 1, 1))
constraints.append(make_dsep_constraint(1, 0, 0))
constraints.append(make_dsep_constraint(1, 0, 1))
constraints.append(make_dsep_constraint(1, 1, 0))
constraints.append(make_dsep_constraint(1, 1, 1))
'''
for x in range(2):
    for a in range(2):
        for b in range(2):
            constraints.append(make_consistency_constraint(x, a, b))
'''
#constraints.append(make_consistency_constraint(0, 0, 0))
#constraints.append(make_consistency_constraint(0, 0, 1))
constraints.append(make_consistency_constraint(0, 1, 0))
constraints.append(make_consistency_constraint(0, 1, 1))
constraints.append(make_consistency_constraint(1, 0, 0))
constraints.append(make_consistency_constraint(1, 0, 1))
constraints.append(make_consistency_constraint(1, 1, 0))
constraints.append(make_consistency_constraint(1, 1, 1))
'''
# 8 consistency constraints
for x in range(2):
    for a in range(2):
        constraints.append(make_consistency_constraint(x, a, 1))
        constraints.append(make_consistency_constraint(x, a, 0))
'''

# Optimization
initial = [1 / 16] * 16
bounds = [(0, 1)] * 16
print(len(constraints))
result = minimize(
    objective,
    initial,
    method='trust-constr',
    constraints=constraints,
    bounds=bounds
)
print()
print("Q(XAB|A#=0)         Q(XAB|A#=1)")
for i in range(4):
  for j in range(4):
    print(f'{result.x[i*4 + j]:.6f}', end="  ")
  print()
print()

print()
print("Q(B=1|A#=1) =", result.fun)
print("Q(A#=1,B=1) =", a_sharp_b(result.x, 1, 1))
print(" P(A=1,B=1) =", dist_xab[3] + dist_xab[7])
print(" Q(A=1,B=1) =", a_b(result.x, 1, 1))
print("    Q(A#=1) =", a_sharp(result.x, 1))
print("    Q(A#=0) =", a_sharp(result.x, 0))


16

Q(XAB|A#=0)         Q(XAB|A#=1)
0.201168  0.017867  0.183540  0.073935  
0.257628  0.017966  0.036612  0.213388  
0.213388  0.036612  0.000278  0.242247  
0.255371  0.000000  0.213388  0.036612  


Q(B=1|A#=1) = 0.9275547591618671
Q(A#=1,B=1) = 0.5661819746479042
 P(A=1,B=1) = 0.25
 Q(A=1,B=1) = 0.26796625799621177
    Q(A#=1) = 1.0
    Q(A#=0) = 1.0


In [5]:
from scipy.optimize import minimize
from functools import partial

xab = []
for x in range(2):
  for a in range(2):
    for b in range(2):
      xab.append(1/8 *(1+ (-1)**(a+b+a*x)/2**0.5))
print(xab)

'''
{'P(A=0,B=0,X=0)': 0.08, 'P(A=0,B=0,X=1)': 0.36, 'P(A=0,B=1,X=0)': 0.04000000000000001, 'P(A=0,B=1,X=1)': 0.06, 'P(A=1,B=0,X=0)': 0.24, 'P(A=1,B=0,X=1)': 0.06, 'P(A=1,B=1,X=0)': 0.04, 'P(A=1,B=1,X=1)': 0.12}
'''
xab = [0.08, 0.04000000000000001, 0.24, 0.04,
       0.36, 0.06, 0.06, 0.12]

def p_index(x, a, b):
  return 4*x + 2*a + b

def q_index(x, a, a_s, b):
  return 8*x + 4*a + 2*a_s + b

def q_a_s(vars, a_s):
  # Q(A# = a_s)
  q = 0
  for x in range(2):
    for a in range(2):
      for b in range(2):
        q += vars[q_index(x, a, a_s, b)]
  return q

def q_a_s_b(vars, a_s, b):
  # Q(A# = a_s, B = b)
  q = 0
  for x in range(2):
    for a in range(2):
      q += vars[q_index(x, a, a_s, b)]
  return q

def q_b_given_a_s(vars, a_s, b):
  # Q(B = b | A# = a_s) = Q(A# = a_s, B = b) / Q(A# = a_s)
  if q_a_s(vars, a_s) == 0:
    return 0
  return q_a_s_b(vars, a_s, b) / q_a_s(vars, a_s)

def q_x_a_s_b(vars, x, a_s, b):
  # Q(X = x, A# = a_s, B = b)
  q = 0
  for a in range(2):
    q += vars[q_index(x, a, a_s, b)]
  return q

def q_x_a_s(vars, x, a_s):
  # Q(X = x, A# = a_s)
  q = 0
  for a in range(2):
    for b in range(2):
      q += vars[q_index(x, a, a_s, b)]
  return q

def q_b_given_x_a_s(vars, x, a_s, b):
  # Q(B = b | X = x, A# = a_s) = Q(X = x, A# = a_s, B = b) / Q(X = x, A# = a_s)
  if q_x_a_s(vars, x, a_s) == 0:
    return 0
  return q_x_a_s_b(vars, x, a_s, b) / q_x_a_s(vars, x, a_s)

def q_a_b_a_s(vars, a, a_s, b):
  # Q(A = a, A# = a_s, B = b)
  q = 0
  for x in range(2):
    q += vars[q_index(x, a, a_s, b)]
  return q

def q_a_b_given_a_s(vars, a, b, a_s):
  # Q(A = a, B = b | A# = a_s) = Q(A# = a_s, A = a, B = b) / Q(A# = a_s)
  if q_a_s(vars, a_s) == 0:
    return 0
  return q_a_b_a_s(vars, a, a_s, b) / q_a_s(vars, a_s)

def d_separation(vars, x, a_s, b):
  # Q(B = b | X = x, A# = a_s) = Q(B = b | A# = a_s)
  return q_b_given_x_a_s(vars, x, a_s, b) - q_b_given_a_s(vars, a_s, b)

def consistency(vars, x, a, b):
  # P(X = x, A = a, B = b) = Q(X = x, A = a, A# = a, B = b)
  return xab[p_index(x, a, b)] - vars[q_index(x, a, a, b)]

def normalization(vars, a_s):
  # Q(A# = a_s) = 1
  return q_a_s(vars, a_s) - 1

def objective(vars):
  # minimize Q(B = 1 | A# = 1)
  return q_b_given_a_s(vars, 1, 1)

def d_separation_constraint(x, a_s, b):
  return {'type':'eq', 'fun': partial(d_separation, x=x, a_s=a_s, b=b)}

def consistency_constraint(x, a, b):
  return {'type':'eq', 'fun': partial(consistency, x=x, a=a, b=b)}

def normalization_constraint(a_s):
  return {'type':'eq', 'fun': partial(normalization, a_s=a_s)}

constraints = []

for x in range(2):
  for a_s in range(2):
    for b in range(2):
      constraints.append(d_separation_constraint(1, a_s, b))

for x in range(2):
  for a in range(2):
    for b in range(2):
      constraints.append(consistency_constraint(x, a, b))

for a_s in range(2):
  constraints.append(normalization_constraint(a_s))

initial = [0.1] * 16

bounds = [(0, 1)] * 16

result = minimize(objective, initial, method='trust-constr', constraints=constraints, bounds=bounds)

print()
print("Q(XAB|A#=0)         Q(XAB|A#=1)")
for i in range(4):
  for j in range(4):
    print(f'{result.x[i*4 + j]:.6f}', end="  ")
  print()
print()

print()
print("     Q(B=1|A#=1) ≥", result.fun)
print("      P(A=1,B=1) =", xab[3] + xab[7])
print(" Q(A=1,B=1|A#=1) =", q_a_b_given_a_s(result.x, 1, 1, 1))
print("         Q(A#=1) =", q_a_s(result.x, 1))
print("         Q(A#=0) =", q_a_s(result.x, 0))

[0.21338834764831843, 0.03661165235168157, 0.03661165235168157, 0.21338834764831843, 0.21338834764831843, 0.03661165235168157, 0.21338834764831843, 0.03661165235168157]


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:81: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is li


Q(XAB|A#=0)         Q(XAB|A#=1)
0.080000  0.040000  0.052225  0.131477  
0.168050  0.075215  0.240000  0.040000  
0.360000  0.060000  0.277975  0.078323  
0.074784  0.141950  0.060000  0.120000  


     Q(B=1|A#=1) ≥ 0.36979956465739744
      P(A=1,B=1) = 0.16
 Q(A=1,B=1|A#=1) = 0.15999999999999998
         Q(A#=1) = 1.0
         Q(A#=0) = 1.0


In [4]:
from scipy.optimize import minimize
from functools import partial

xab = []
for x in range(2):
  for a in range(2):
    for b in range(2):
      xab.append(1/8 *(1+ (-1)**(a+b+a*x)/2**0.5))
print(xab)

def p_index(x, a, b):
  return 4*x + 2*a + b

def q_index(x, a, a_s, b):
  return 8*x + 4*a + 2*a_s + b

def q_a_s(vars, a_s):
  # Q(A# = a_s)
  q = 0
  for x in range(2):
    for a in range(2):
      for b in range(2):
        q += vars[q_index(x, a, a_s, b)]
  return q

def q_a_s_b(vars, a_s, b):
  # Q(A# = a_s, B = b)
  q = 0
  for x in range(2):
    for a in range(2):
      q += vars[q_index(x, a, a_s, b)]
  return q

def q_b_given_a_s(vars, a_s, b):
  # Q(B = b | A# = a_s) = Q(A# = a_s, B = b) / Q(A# = a_s)
  if q_a_s(vars, a_s) == 0:
    return 0
  return q_a_s_b(vars, a_s, b) / q_a_s(vars, a_s)

def q_x_a_s_b(vars, x, a_s, b):
  # Q(X = x, A# = a_s, B = b)
  q = 0
  for a in range(2):
    q += vars[q_index(x, a, a_s, b)]
  return q

def q_x_a_s(vars, x, a_s):
  # Q(X = x, A# = a_s)
  q = 0
  for a in range(2):
    for b in range(2):
      q += vars[q_index(x, a, a_s, b)]
  return q

def q_b_given_x_a_s(vars, x, a_s, b):
  # Q(B = b | X = x, A# = a_s) = Q(X = x, A# = a_s, B = b) / Q(X = x, A# = a_s)
  if q_x_a_s(vars, x, a_s) == 0:
    return 0
  return q_x_a_s_b(vars, x, a_s, b) / q_x_a_s(vars, x, a_s)

def q_a_b_a_s(vars, a, a_s, b):
  # Q(A = a, A# = a_s, B = b)
  q = 0
  for x in range(2):
    q += vars[q_index(x, a, a_s, b)]
  return q

def q_a_b_given_a_s(vars, a, b, a_s):
  # Q(A = a, B = b | A# = a_s) = Q(A# = a_s, A = a, B = b) / Q(A# = a_s)
  if q_a_s(vars, a_s) == 0:
    return 0
  return q_a_b_a_s(vars, a, a_s, b) / q_a_s(vars, a_s)

def d_separation(vars, x, a_s, b):
  # Q(B = b | X = x, A# = a_s) = Q(B = b | A# = a_s)
  return q_b_given_x_a_s(vars, x, a_s, b) - q_b_given_a_s(vars, a_s, b)

def consistency(vars, x, a, b):
  # P(X = x, A = a, B = b) = Q(X = x, A = a, A# = a, B = b)
  return xab[p_index(x, a, b)] - vars[q_index(x, a, a, b)]

def normalization(vars, a_s):
  # Q(A# = a_s) = 1
  return q_a_s(vars, a_s) - 1

def objective(vars):
  # minimize Q(B = 1 | A# = 1)
  return q_b_given_a_s(vars, 1, 1)

def d_sep2(vars, a_s, b):
  return q_b_given_x_a_s(vars, 1, a_s, b) - q_b_given_x_a_s(vars, 0, a_s, b)

def d_sep2_constraint(a_s, b):
  return {'type':'eq', 'fun': partial(d_sep2, a_s=a_s, b=b)}

def d_separation_constraint(x, a_s, b):
  return {'type':'eq', 'fun': partial(d_separation, x=x, a_s=a_s, b=b)}

def consistency_constraint(x, a, b):
  return {'type':'eq', 'fun': partial(consistency, x=x, a=a, b=b)}

def normalization_constraint(a_s):
  return {'type':'eq', 'fun': partial(normalization, a_s=a_s)}

constraints = []
'''
for x in range(2):
  for a_s in range(2):
    for b in range(2):
      constraints.append(d_separation_constraint(1, a_s, b))'''

for a_s in range(2):
  for b in range(2):
    constraints.append(d_sep2_constraint(a_s, b))

for x in range(2):
  for a in range(2):
    for b in range(2):
      constraints.append(consistency_constraint(x, a, b))

for a_s in range(2):
  constraints.append(normalization_constraint(a_s))

initial = [0.1] * 16

bounds = [(0, 1)] * 16

result = minimize(objective, initial, method='trust-constr', constraints=constraints, bounds=bounds)

print()
print("Q(XAB|A#=0)         Q(XAB|A#=1)")
for i in range(4):
  for j in range(4):
    print(f'{result.x[i*4 + j]:.6f}', end="  ")
  print()
print()

print()
print("     Q(B=1|A#=1) ≥", result.fun)
print("      P(A=1,B=1) =", xab[3] + xab[7])
print(" Q(A=1,B=1|A#=1) =", q_a_b_given_a_s(result.x, 1, 1, 1))
print("         Q(A#=1) =", q_a_s(result.x, 1))
print("         Q(A#=0) =", q_a_s(result.x, 0))

[0.21338834764831843, 0.03661165235168157, 0.03661165235168157, 0.21338834764831843, 0.21338834764831843, 0.03661165235168157, 0.21338834764831843, 0.03661165235168157]


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:81: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_trustregion_constr/equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is li


Q(XAB|A#=0)         Q(XAB|A#=1)
0.213388  0.036612  0.220386  0.046289  
0.073027  0.198748  0.036612  0.213388  
0.213388  0.036612  0.027021  0.206304  
0.049121  0.179104  0.213388  0.036612  


     Q(B=1|A#=1) ≥ 0.5025930638533961
      P(A=1,B=1) = 0.25
 Q(A=1,B=1|A#=1) = 0.25
         Q(A#=1) = 1.0
         Q(A#=0) = 1.0


In [2]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 58.8 MB/s eta 0:00:00


In [ ]:
from gurobipy import Model, GRB, quicksum

xab = []
for x in range(2):
    for a in range(2):
        for b in range(2):
            xab.append(1 / 8 * (1 + (-1)**(a + b + a * x) / 2**0.5))

def p_index(x, a, b): return 4 * x + 2 * a + b
def q_index(x, a, a_s, b): return 8 * x + 4 * a + 2 * a_s + b

model = Model("linearize_q")
model.Params.OutputFlag = 0  # silence solver output

# Create original vars q_hat[x,a,a_s,b] which will later be scaled
q_hat = model.addVars(2, 2, 2, 2, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="q_hat") # Increased upper bound
t = model.addVar(lb=1e-6, vtype=GRB.CONTINUOUS, name="t")  # scaling variable

# Define scaled vars: q[x,a,a_s,b] = q_hat[x,a,a_s,b] / t
# But we avoid division by defining constraints using scaled vars directly
# All constraints and objectives will be in terms of q_hat and t

# Consistency: P(X,A,B) = Q(X,A,A#=A,B)
for x in range(2):
    for a in range(2):
        for b in range(2):
            model.addConstr(q_hat[x, a, a, b] == xab[p_index(x, a, b)] * t)

# D-separation: Q(B | X, A#) = Q(B | A#)
# Q(B, X, A#) / Q(X, A#) = Q(B, A#) / Q(A#)
# Q(B, X, A#) * Q(A#) = Q(B, A#) * Q(X, A#)
# (Sum_a q_hat(x, a, a_s, b)/t) * (Sum_x,a,b q_hat(x, a, a_s, b)/t) = (Sum_x,a q_hat(x, a, a_s, b)/t) * (Sum_a,b q_hat(x, a, a_s, b)/t)
# Sum_a q_hat(x, a, a_s, b) * Sum_x,a,b q_hat(x, a, a_s, b) = Sum_x,a q_hat(x, a, a_s, b) * Sum_a,b q_hat(x, a, a_s, b)
for x_val in range(2): # Renamed loop variable to avoid conflict
    for a_s_val in range(2): # Renamed loop variable
        for b_val in range(2): # Renamed loop variable
            # Q(B, X, A#) term: Sum_a q_hat(x, a, a_s, b)
            q_b_x_as_hat = quicksum(q_hat[x_val, a, a_s_val, b_val] for a in range(2))

            # Q(A#) term: Sum_x,a,b q_hat(x, a, a_s, b) which is equal to t due to normalization
            q_as_hat = quicksum(q_hat[x_val_inner, a_val_inner, a_s_val, b_val_inner]
                                for x_val_inner in range(2)
                                for a_val_inner in range(2)
                                for b_val_inner in range(2)) # This sum is equal to t

            # Q(B, A#) term: Sum_x,a q_hat(x, a, a_s, b)
            q_b_as_hat = quicksum(q_hat[x, a, a_s_val, b_val] for x in range(2) for a in range(2))

            # Q(X, A#) term: Sum_a,b q_hat(x, a, a_s, b)
            q_x_as_hat = quicksum(q_hat[x_val, a, a_s_val, b] for a in range(2) for b in range(2))

            # Add the linearized d-separation constraint
            model.addConstr(q_b_x_as_hat * q_as_hat == q_b_as_hat * q_x_as_hat)


# Normalization: Q(A# = a_s) sums to t
for a_s in range(2):
    model.addConstr(quicksum(q_hat[x, a, a_s, b] for x in range(2) for a in range(2) for b in range(2)) == t)

# Objective: minimize Q(B=1 | A#=1) = Q_hat(B=1, A#=1) / t
numerator = quicksum(q_hat[x, a, 1, 1] for x in range(2) for a in range(2))
model.setObjective(numerator, GRB.MINIMIZE) # Objective is to minimize numerator; minimizing numerator/t is equivalent since t > 0

model.optimize()

# Recover the true Q by normalizing
true_q = {}
# Add a small epsilon to t.X for normalization to avoid division by zero if t.X is extremely close to 0
epsilon = 1e-9 # Define a small epsilon
denominator_t = t.X if t.X > epsilon else epsilon # Use epsilon if t.X is too small

for x in range(2):
    for a in range(2):
        for a_s in range(2):
            for b in range(2):
                true_q[(x, a, a_s, b)] = q_hat[x, a, a_s, b].X / denominator_t # Use the adjusted denominator

print("\nQ(XAB|A#=0)         Q(XAB|A#=1)")
for x in range(2):
    for a in range(2):
        for b in range(2):
            print(f"{true_q[x, a, 0, b]:.6f}", end="  ")
        print("   ", end="")
        for b in range(2):
            print(f"{true_q[x, a, 1, b]:.6f}", end="  ")
        print()
print()

# Recalculate the denominator for Q(B=1|A#=1) using the normalized true_q values
# Add epsilon here as well for safety, although it should be non-zero if t.X was > epsilon
denominator_q_b1_a1 = sum(true_q[x, a, 1, b] for x in range(2) for a in range(2) for b in range(2))
if denominator_q_b1_a1 < epsilon:
    q_b1_a1 = float('inf') # Or handle as appropriate for a zero denominator in the final result
else:
    q_b1_a1 = sum(true_q[x, a, 1, 1] for x in range(2) for a in range(2)) / denominator_q_b1_a1

print(f"Q(B=1|A#=1) = {q_b1_a1}")


Q(XAB|A#=0)         Q(XAB|A#=1)
0.000000  0.000000     0.000000  0.000000  
0.000000  0.000000     0.000000  0.000000  
0.000000  0.000000     0.000000  0.000000  
0.000000  0.000000     0.000000  0.000000  

Q(B=1|A#=1) = inf
